In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [ ]:
con = sqlite3.connect("F:\Data Analysis Projects\Amazon\database.sqlite")

In [ ]:
type(con)

In [ ]:
df = pd.read_sql_query("Select * from Reviews",con)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
pd.read_sql_query("Select * from Reviews Limit 10",con)

In [ ]:
pd.read_sql_query("SELECT * FROM sqlite_master",con)

In [ ]:
# Perform Sentiment Analysis on Data

In [ ]:
from textblob import TextBlob

In [ ]:
text = df["Summary"][0]
text

In [ ]:
TextBlob(text).sentiment.polarity

In [ ]:
# if there are null values use try and except

In [ ]:
polarity = []
 

for i in df["Summary"]:
    try:
        polarity.append(TextBlob(i).sentiment.polarity)
    except:
        polarity.append(0)

In [ ]:
len(polarity)

In [ ]:
data = df.copy()

In [ ]:
data["polarity"] = polarity

In [ ]:
data.head()

In [ ]:
# Perform EDA for positive sentences

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
stopwords = STOPWORDS

In [ ]:
type(stopwords)

In [ ]:
data_positive = data[data["polarity"]>0]

In [ ]:
data_positive.shape

In [ ]:
total_text = ''.join(data_positive["Summary"])

In [ ]:
total_text[:5000]

In [ ]:
len(total_text)

In [ ]:
import re


In [ ]:
# Here ^ is except. It tells except a-zA-Z substitute it with space

total_text = re.sub('[^a-zA-Z]',' ',total_text)

In [ ]:
total_text

In [ ]:
# It tells if we have more than one extra space replace it with one space

total_text = re.sub(' +',' ',total_text)
total_text

In [ ]:
wordcloud = WordCloud(width = 400, height = 200, stopwords = stopwords).generate(total_text)
plt.figure(figsize = (15,5))
plt.imshow(wordcloud)
plt.axis("off")

In [ ]:
# Perform EDA for negative sentences

In [ ]:
data_negative = data[data["polarity"]<0]

In [ ]:
total_text2 = ''.join(data_negative["Summary"])

In [ ]:
total_text2 = re.sub('[^a-zA-Z]',' ',total_text2)
total_text2

In [ ]:
total_text2 = re.sub(' +',' ',total_text2)
total_text2

In [ ]:
wordcloud2 = WordCloud(width = 1000, height = 500, stopwords = stopwords).generate(total_text2)
plt.figure(figsize = (15,5))
plt.imshow(wordcloud2)
plt.axis("off")

In [ ]:
# Analysis, to what type of user Amazon recommend more product

In [ ]:
df.head()

In [ ]:
df["UserId"].nunique()

In [ ]:
raw = df.groupby("UserId").agg({"Summary":"count","Text":"count","Score":"mean","ProductId":"count"}).sort_values(by = "ProductId",ascending = False)

In [ ]:
raw.columns = ["Number of Summaries","Number of Text","Avg Score","Number of Product Purchased"]

In [ ]:
raw.head()

In [ ]:
user_index10 = raw.index[0:10]
number_10 = raw["Number of Product Purchased"][0:10]

In [ ]:
plt.figure(figsize = (20,5))
plt.bar(user_index10, number_10, label ="Most Recommended Users", width = 0.8 )
plt.xlabel("UserId")
plt.ylabel("Number of Product Purchased")
plt.xticks(rotation = "vertical")

In [ ]:
# Analyze the length of the comment, whether customer are giving lengthy comment or short one

In [ ]:
final = df[0:2000]

In [ ]:
final.head()

In [ ]:
final.isnull().sum()

In [ ]:
final.duplicated().sum()

In [ ]:
# We have to split the text on the basis of space other wise while calculating length it count spaces also

len(final["Text"][0].split(' '))

In [ ]:
def cal_len(text):
    return len(text.split(' '))

In [ ]:
# Now we have to map or apply the function

final["Text_Length"] = final["Text"].apply(cal_len)

In [ ]:
final.head()

In [ ]:
import plotly.express as px

In [ ]:
px.box(final, y = "Text_Length")

In [ ]:
sns.countplot(final["Score"])

In [ ]:
# Convert all the words in lower letter because if we do not, Care or care can be different

In [ ]:
final["Text"] = final["Text"].str.lower()

In [ ]:
final.head()

In [ ]:
data = final["Text"][180]
data

In [ ]:
punctuation = '''?,\/@$#%!-'''
no_punc = ''

for char in data:
    if char not in punctuation:
        no_punc = no_punc + char
        
no_punc


In [ ]:
final.head()

In [ ]:
import string

punctuations = string.punctuation

def remove_punc(texts):
    no_punc = ''
    for char in texts:
         if char not in punctuations:
                no_punc = no_punc + char
    return no_punc
        

    

In [ ]:
final["Text"] = final["Text"].apply(remove_punc)

In [ ]:
final["Text"][164]

In [ ]:
final.head()

In [ ]:
data = final["Text"][180]
data

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
re = [word for word in data.split(' ') if word not in set(stopwords.words("english"))]
re

In [ ]:
strs = ''
for wd in re:
    strs = strs + wd
    strs = strs + " "
strs

In [ ]:
def remove_stopwords(review):
    return' '.join([word for word in data.split(' ') if word not in set(stopwords.words("english"))])

In [ ]:
final["Text"] = final["Text"].apply(remove_stopwords)

In [ ]:
final["Text"][100]

In [ ]:
df["Text"].str.contains("https?").sum()

In [ ]:
pd.set_option("display.max_rows",None)
df["Text"].str.contains("https?")

In [ ]:
review = df["Text"][41409]
review

In [ ]:
# compile used to find patterns and  w+ shows more than one word characters

In [ ]:
import re

url_pattern = re.compile(r'href=https.\w+')
url_pattern.sub(r'', review)